In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

# Sample

In [1]:
outline_ = {
  "slides": [
    {
      "slide_number": 1,
      "title": "AI in Marketing: The Future is Now",
      "content": "- Why AI matters in marketing\n- Setting the stage for transformation",
      "transcript": "Welcome to 'AI in Marketing: The Future is Now'. In this opening slide, we set the stage for exploring how artificial intelligence is not just a tool but a transformative force in the marketing industry. We will discuss why AI matters, the rapid pace of innovation, and how it is reshaping customer engagement."
    },
    {
      "slide_number": 2,
      "title": "What is AI in Marketing?",
      "content": "- Definition\n- Scope of use in marketing",
      "transcript": "Artificial Intelligence in marketing refers to the use of machine learning algorithms and data-driven models to automate decisions, personalize customer experiences, and optimize marketing campaigns. It encompasses everything from recommendation engines to chatbots, targeting strategies, and predictive analytics."
    },
    {
      "slide_number": 3,
      "title": "Five-Step Customer Journey",
      "content": "- Awareness\n- Consideration\n- Conversion\n- Retention\n- Advocacy",
      "transcript": "The customer journey can be broken down into five essential stages. Awareness is where potential customers first learn about your brand. Consideration is when they start evaluating your product or service. Conversion is the moment they decide to buy. Retention focuses on keeping customers engaged and satisfied. Advocacy is when customers promote your brand to others. AI can enhance each stage through personalized touchpoints."
    },
    {
      "slide_number": 4,
      "title": "AI Tools: A Comparison",
      "content": "- Tool A: High accuracy, higher cost\n- Tool B: Moderate accuracy, affordable",
      "transcript": "Let's compare two AI marketing tools. Tool A offers high accuracy in targeting and recommendations but comes at a significantly higher cost. Tool B is more affordable, with moderate accuracy and fewer advanced features. Choosing the right tool depends on your budget, marketing goals, and the complexity of your campaigns."
    },
    {
      "slide_number": 5,
      "title": "Real-World Success Stories",
      "content": "- Retail: Personalized shopping experiences\n- Hospitality: Dynamic pricing models",
      "transcript": "In retail, companies like Amazon have successfully used AI to deliver highly personalized shopping experiences, recommending products based on browsing and purchase history. In the hospitality industry, AI-driven dynamic pricing models help hotels and airlines optimize rates in real-time, maximizing revenue while maintaining competitive pricing."
    },
    {
      "slide_number": 6,
      "title": "Benefits of AI in Marketing",
      "content": "- Efficiency through automation\n- Improved personalization\n- Data-driven decisions\n- Cost optimization",
      "transcript": "AI brings multiple benefits to marketing. Efficiency is achieved by automating repetitive tasks. Personalization is improved through real-time customer data analysis. Decision-making becomes more precise and timely thanks to predictive analytics. Finally, marketing budgets can be optimized by allocating resources where they will have the most impact."
    },
    {
      "slide_number": 7,
      "title": "Challenges and Risks",
      "content": "- Data privacy issues\n- Algorithmic bias\n- High adoption costs\n- Skill shortages",
      "transcript": "Despite its benefits, AI in marketing comes with challenges. Data privacy concerns are growing due to the massive amount of personal data collected. Algorithmic bias can lead to unfair targeting or discrimination. High adoption costs can be a barrier, especially for small businesses. Additionally, there is a shortage of skilled professionals who can design, implement, and maintain AI systems."
    },
    {
      "slide_number": 8,
      "title": "Key Takeaways",
      "content": "- AI is transforming every stage of marketing\n- Balance innovation with ethics\n- Invest in skills and infrastructure",
      "transcript": "In summary, AI is revolutionizing every stage of the marketing process. However, it is important to balance innovation with ethical considerations, ensuring that technologies are used responsibly. Businesses should also invest in the skills and infrastructure needed to leverage AI effectively."
    }
  ]
}


# Schema

In [2]:
from typing import List, Tuple, Literal, Optional, Dict
from pydantic import BaseModel, Field

Rect = Tuple[int, int, int, int]  # x,y,w,h

class Slot(BaseModel):
    id: str
    kind: Literal["text","image","video","shape","presenter"]
    rect: Rect
    z: int = 0
    payload: Optional[str] = None      # text hoặc path/URL
    style: Dict = Field(default_factory=dict)
    anim: Dict = Field(default_factory=dict)

class Scene(BaseModel):
    scene_id: str
    slide_number: int
    layout: str
    presenter_mode: Literal["off","pip_left","pip_right","half_frame"] = "pip_right"
    slots: List[Slot] = []
    start: float = 0.0          # điền ở bước 8
    duration: float = 0.0       # lấy từ media ở bước 8
    media: Dict = {}            # {"presenter_video":"s01-presenter.mp4", "audio":"s01-audio.mp3", "captions":"s01.vtt"}
    assets_spec: Dict = {}      # {"image_keywords":[...], ...}

class ScenePlan(BaseModel):
    version: str = "1.0"
    fps: int = 30
    resolution: Tuple[int,int] = (1920,1080)
    scenes: List[Scene] = []
    meta: Dict = {}             # {"speaker_wpm":140, "theme":"light"}



# Prompt

In [33]:
from langchain.prompts import ChatPromptTemplate

INIT_SCENES_PROMPT = """
Bạn là một chuyên gia trong việc xây dựng kịch bản video E-Learning.
Bạn sẽ nhận được một Outline gồm các ý chính sẽ được trình bày trong video. Mỗi Ý trong Outline sẽ bao gồm các slide, mỗi slide bao gồm:
- Title: tiêu đề của video
- Content: các nội dung chính thuộc tiêu đề trên.
- Transcript: lời thoại dành cho presenter, lời thoại sẽ mang nội dung đầy đủ và được diễn giải của content tuy nhiên sẽ không hiển thị lên video.
Bạn sẽ cần tạo một số thuộc tính sau cho scene theo JSON format:
- scene_id: thường bắt đầu bằng chữ s và gắn liền với slide number (ví dụ: 1,2,3,..)
- slide_number: thứ tự của slide chứa nội dung tương ứng với scene (ví dụ: s01, s02,s03,..)
- layout: dựa vào nội dung và bộ layout template mà bạn được cung cấp bên dưới, hãy đề xuất layout phù hợp với scene này để có thể hiển thị đầy đủ nội dung và assets(nếu có).
Outline:
{outline}
Layout Template:
{layout_template}
Tuyệt đối không trả về thêm bất cứ output nào ngoài list scene kể cả ```json 
"""
IS_prompt = ChatPromptTemplate.from_template(INIT_SCENES_PROMPT)

In [6]:
import os 
template_root = "D:/ThienPV/code/demo/src/templates/scenes"
template_paths = os.listdir(path = template_root)
len(template_paths)

11

In [7]:
import json 
templates = []
for path in template_paths:
    path = os.path.join(template_root, path)
    with open(path, "r", encoding="utf-8") as fp:
        template = json.load(fp)
        templates.append(template)

# Agent

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [34]:
init_scene_chain = IS_prompt | llm

In [35]:
out = init_scene_chain.invoke(
    {
        "outline": outline_,
        "layout_template": str(templates)
    }
)
out

AIMessage(content='```json\n[\n  {\n    "scene_id": "s01",\n    "slide_number": 1,\n    "layout": "L01_HeroTitle"\n  },\n  {\n    "scene_id": "s02",\n    "slide_number": 2,\n    "layout": "L07_Definition_Card"\n  },\n  {\n    "scene_id": "s03",\n    "slide_number": 3,\n    "layout": "L08_Stepper_Process_3to5"\n  },\n  {\n    "scene_id": "s04",\n    "slide_number": 4,\n    "layout": "L06_SplitCompare_AvsB"\n  },\n  {\n    "scene_id": "s05",\n    "slide_number": 5,\n    "layout": "L05_ImageFocus_Right"\n  },\n  {\n    "scene_id": "s06",\n    "slide_number": 6,\n    "layout": "L02_TitleBullets_1Col"\n  },\n  {\n    "scene_id": "s07",\n    "slide_number": 7,\n    "layout": "L02_TitleBullets_1Col"\n  },\n  {\n    "scene_id": "s08",\n    "slide_number": 8,\n    "layout": "L10_Outro_CTA"\n  }\n]\n```', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-

In [36]:
print(out.content)

```json
[
  {
    "scene_id": "s01",
    "slide_number": 1,
    "layout": "L01_HeroTitle"
  },
  {
    "scene_id": "s02",
    "slide_number": 2,
    "layout": "L07_Definition_Card"
  },
  {
    "scene_id": "s03",
    "slide_number": 3,
    "layout": "L08_Stepper_Process_3to5"
  },
  {
    "scene_id": "s04",
    "slide_number": 4,
    "layout": "L06_SplitCompare_AvsB"
  },
  {
    "scene_id": "s05",
    "slide_number": 5,
    "layout": "L05_ImageFocus_Right"
  },
  {
    "scene_id": "s06",
    "slide_number": 6,
    "layout": "L02_TitleBullets_1Col"
  },
  {
    "scene_id": "s07",
    "slide_number": 7,
    "layout": "L02_TitleBullets_1Col"
  },
  {
    "scene_id": "s08",
    "slide_number": 8,
    "layout": "L10_Outro_CTA"
  }
]
```


In [38]:
out_ = [
  {
    "scene_id": "s01",
    "slide_number": 1,
    "layout": "L01_HeroTitle"
  },
  {
    "scene_id": "s02",
    "slide_number": 2,
    "layout": "L07_Definition_Card"
  },
  {
    "scene_id": "s03",
    "slide_number": 3,
    "layout": "L08_Stepper_Process_3to5"
  },
  {
    "scene_id": "s04",
    "slide_number": 4,
    "layout": "L06_SplitCompare_AvsB"
  },
  {
    "scene_id": "s05",
    "slide_number": 5,
    "layout": "L05_ImageFocus_Right"
  },
  {
    "scene_id": "s06",
    "slide_number": 6,
    "layout": "L02_TitleBullets_1Col"
  },
  {
    "scene_id": "s07",
    "slide_number": 7,
    "layout": "L02_TitleBullets_1Col"
  },
  {
    "scene_id": "s08",
    "slide_number": 8,
    "layout": "L10_Outro_CTA"
  }
]

In [41]:
for idx, scene in enumerate(out_):
    print(idx)
    print(scene)
    print(outline_["slides"][idx])

0
{'scene_id': 's01', 'slide_number': 1, 'layout': 'L01_HeroTitle'}
{'slide_number': 1, 'title': 'AI in Marketing: The Future is Now', 'content': '- Why AI matters in marketing\n- Setting the stage for transformation', 'transcript': "Welcome to 'AI in Marketing: The Future is Now'. In this opening slide, we set the stage for exploring how artificial intelligence is not just a tool but a transformative force in the marketing industry. We will discuss why AI matters, the rapid pace of innovation, and how it is reshaping customer engagement."}
1
{'scene_id': 's02', 'slide_number': 2, 'layout': 'L07_Definition_Card'}
{'slide_number': 2, 'title': 'What is AI in Marketing?', 'content': '- Definition\n- Scope of use in marketing', 'transcript': 'Artificial Intelligence in marketing refers to the use of machine learning algorithms and data-driven models to automate decisions, personalize customer experiences, and optimize marketing campaigns. It encompasses everything from recommendation engin

In [42]:
for it in outline_["slides"]:
    print(it["title"])
    print(it["content"])

AI in Marketing: The Future is Now
- Why AI matters in marketing
- Setting the stage for transformation
What is AI in Marketing?
- Definition
- Scope of use in marketing
Five-Step Customer Journey
- Awareness
- Consideration
- Conversion
- Retention
- Advocacy
AI Tools: A Comparison
- Tool A: High accuracy, higher cost
- Tool B: Moderate accuracy, affordable
Real-World Success Stories
- Retail: Personalized shopping experiences
- Hospitality: Dynamic pricing models
Benefits of AI in Marketing
- Efficiency through automation
- Improved personalization
- Data-driven decisions
- Cost optimization
Challenges and Risks
- Data privacy issues
- Algorithmic bias
- High adoption costs
- Skill shortages
Key Takeaways
- AI is transforming every stage of marketing
- Balance innovation with ethics
- Invest in skills and infrastructure


In [ ]:
def init_scene_from_outline (outline: Dict)-> Scene:
    

In [ ]:
def init_scene_from_slide(slide) -> Scene: ...
def build_slots(scene, slide, layout_lib, theme) -> None: ...
def decide_presenter_mode(scene) -> str: ...
def synthesize_presenter(scene, transcript) -> Dict: ...  # returns media dict
def derive_assets_spec(slide) -> Dict: ...
def arrange_timeline(scenes, media_probe) -> None: ...    # fill start/duration
def quick_adjust(scene) -> None: ...